In [1]:
import pandas as pd
import numpy as np
import folium
import json
from folium import plugins
import os
from sklearn.preprocessing import StandardScaler

- 생활인구 관내이동 데이터 전처리

In [2]:
df = pd.read_csv("서울 생활인구.csv", encoding='utf-8')
# 20대부터 40대까지의 남자 유동인구 데이터만 가져옵니다.
cols = {
    '남자20세부터24세생활인구수':2024,
    '남자25세부터29세생활인구수':2529,
    '남자30세부터34세생활인구수':3034,
    '남자35세부터39세생활인구수':3539,
    '남자40세부터44세생활인구수':4044,
    '남자45세부터49세생활인구수':4549    
}

df.rename(columns=cols, inplace=True)  # 컬럼명 재설정

for i in [2024,2529,3034,3539,4044,4549]:
    df[i] = df[i].astype(float) 

display(df)

,기준일ID,시간대구분,행정동코드,행정동명,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,2024,2529,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20230524,0,11110515,서울특별시 종로구 청운효자동,15647.1864,691.0474,440.3371,558.2620,508.1378,419.0239,...,440.2464,540.9623,617.9661,725.5513,864.5871,631.7088,653.5150,496.0182,361.7608,1216.0881
1,20230524,0,11110530,서울특별시 종로구 사직동,12993.8830,363.7416,161.6629,194.6075,327.0923,412.6600,...,452.1665,517.9600,734.4700,546.3135,668.9874,518.9216,496.9823,463.8421,298.8675,966.2442
2,20230524,0,11110540,서울특별시 종로구 삼청동,3698.2805,83.7208,64.1501,309.4145,218.2705,93.4354,...,107.4392,105.8846,128.8961,198.9166,175.4269,92.7180,124.8407,130.3561,115.5340,294.9591
3,20230524,0,11110550,서울특별시 종로구 부암동,14041.0218,437.4616,339.2018,517.3964,434.9488,395.0124,...,444.1662,340.5095,542.0175,632.5932,794.9249,600.6828,591.5199,479.9927,400.9613,1076.3031
4,20230524,0,11110560,서울특별시 종로구 평창동,18966.8192,594.7278,430.1859,571.4202,618.2474,498.9935,...,455.9894,448.3025,689.6130,766.6149,857.8774,814.1352,1108.7229,822.2288,638.5887,1700.9013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10171,20230524,23,11740650,서울특별시 강동구 성내2동,25858.1278,1084.6708,376.2586,480.4422,543.4767,925.4540,...,1400.7770,1180.1005,1189.3127,888.2065,1061.0983,946.0755,1130.0915,946.0553,787.3013,1745.7203
10172,20230524,23,11740660,서울특별시 강동구 성내3동,27749.9172,844.5820,507.6774,781.7230,884.7026,965.4784,...,1201.4170,1081.0195,1084.0776,1187.6209,1144.6856,1063.8577,1249.4995,1206.1633,750.3297,1993.2908
10173,20230524,23,11740685,서울특별시 강동구 길동,59243.1878,1885.5410,911.7832,1213.6083,1427.5202,1989.9143,...,2348.0914,2394.7430,2434.9731,2208.3267,2532.9063,2320.9975,2622.8622,2407.2742,1883.6969,4710.4882
10174,20230524,23,11740690,서울특별시 강동구 둔촌1동,4959.0392,209.4570,158.7470,339.9146,95.3699,111.6044,...,133.9043,117.7690,157.4859,239.5330,203.7058,190.8291,183.5997,195.3691,137.4098,404.8526


In [3]:
df['풋살인구'] = df[2024] + df[2529] + df[3034] + df[3539] + df[4044] + df[4549]

In [4]:
columns_to_keep = ['기준일ID', '시간대구분', '행정동코드', '행정동명', '총생활인구수','풋살인구']
df = df[columns_to_keep]
display(df)

,기준일ID,시간대구분,행정동코드,행정동명,총생활인구수,풋살인구
0,20230524,0,11110515,서울특별시 종로구 청운효자동,15647.1864,3111.9341
1,20230524,0,11110530,서울특별시 종로구 사직동,12993.8830,3097.7317
2,20230524,0,11110540,서울특별시 종로구 삼청동,3698.2805,892.9059
3,20230524,0,11110550,서울특별시 종로구 부암동,14041.0218,2585.9033
4,20230524,0,11110560,서울특별시 종로구 평창동,18966.8192,3302.8349
...,...,...,...,...,...,...
10171,20230524,23,11740650,서울특별시 강동구 성내2동,25858.1278,5888.3954
10172,20230524,23,11740660,서울특별시 강동구 성내3동,27749.9172,6472.6540
10173,20230524,23,11740685,서울특별시 강동구 길동,59243.1878,12804.5989
10174,20230524,23,11740690,서울특별시 강동구 둔촌1동,4959.0392,921.6160


In [5]:
#시간대 별, 행정동 별 유동인구의 합을 구합니다.
result1 = df.groupby(['시간대구분', '행정동명'])['풋살인구'].sum()
df2 = pd.DataFrame(result1)
display(df2)

풋살인구
시간대구분 행정동명                      
0     서울특별시 강남구 개포1동   1683.4231
      서울특별시 강남구 개포2동   6343.4255
      서울특별시 강남구 개포3동   3611.9645
      서울특별시 강남구 개포4동   3685.4781
      서울특별시 강남구 논현1동  11753.3074
...                          ...
23    서울특별시 중랑구 상봉2동   7045.9530
      서울특별시 중랑구 신내1동   6602.2070
      서울특별시 중랑구 신내2동   3487.4028
      서울특별시 중랑구 중화1동   3285.2510
      서울특별시 중랑구 중화2동   5671.8913

[10176 rows x 1 columns]

In [6]:
#데이터 프레임을 뽑기 위해 해당 index를 컬럼으로 복사합니다.(reset_index를 썼으면 한 방에 해결되는 것을 나중에 알았습니다.)
index = []
b = []
for i in df2.index:
    index.append(i[0])

for j in index:
    b.append(int(j))

df2['시간대'] = b

index = []
b = []
for i in df2.index:
    index.append(i[1])

for j in index:
    b.append(str(j))

df2['행정동명'] = b
display(df2)

풋살인구  시간대            행정동명
시간대구분 행정동명                                           
0     서울특별시 강남구 개포1동   1683.4231    0  서울특별시 강남구 개포1동
      서울특별시 강남구 개포2동   6343.4255    0  서울특별시 강남구 개포2동
      서울특별시 강남구 개포3동   3611.9645    0  서울특별시 강남구 개포3동
      서울특별시 강남구 개포4동   3685.4781    0  서울특별시 강남구 개포4동
      서울특별시 강남구 논현1동  11753.3074    0  서울특별시 강남구 논현1동
...                          ...  ...             ...
23    서울특별시 중랑구 상봉2동   7045.9530   23  서울특별시 중랑구 상봉2동
      서울특별시 중랑구 신내1동   6602.2070   23  서울특별시 중랑구 신내1동
      서울특별시 중랑구 신내2동   3487.4028   23  서울특별시 중랑구 신내2동
      서울특별시 중랑구 중화1동   3285.2510   23  서울특별시 중랑구 중화1동
      서울특별시 중랑구 중화2동   5671.8913   23  서울특별시 중랑구 중화2동

[10176 rows x 3 columns]

In [7]:
#컬럼들만 뽑아 데이터 프레임을 만듭니다.
#df2.to_csv('생활인구_groupby.csv', index=False, encoding = 'cp949')

In [8]:
#완성된 유동인구 데이터 프레임
df = pd.read_csv('생활인구_groupby.csv', encoding = 'cp949')
display(df)

,풋살인구,시간대,행정동명
0,1683.4231,0,서울특별시 강남구 개포1동
1,6343.4255,0,서울특별시 강남구 개포2동
2,3611.9645,0,서울특별시 강남구 개포3동
3,3685.4781,0,서울특별시 강남구 개포4동
4,11753.3074,0,서울특별시 강남구 논현1동
...,...,...,...
10171,7045.9530,23,서울특별시 중랑구 상봉2동
10172,6602.2070,23,서울특별시 중랑구 신내1동
10173,3487.4028,23,서울특별시 중랑구 신내2동
10174,3285.2510,23,서울특별시 중랑구 중화1동


- 인구밀도 데이터 전처리

In [9]:
df = pd.read_csv('인구밀도야.csv', encoding='utf-8')
dense = df[['행정동명', '인구밀도']]
display(dense)

,행정동명,인구밀도
0,서울특별시 종로구 사직동,7606
1,서울특별시 종로구 삼청동,1773
2,서울특별시 종로구 부암동,4201
3,서울특별시 종로구 평창동,2013
4,서울특별시 종로구 무악동,22367
...,...,...
421,서울특별시 강동구 천호2동,21545
422,서울특별시 강동구 길동,20910
423,서울특별시 강동구 강일동,14627
424,서울특별시 강동구 상일제1동,14784


In [10]:
#보다 정교화된 시각화를 위해 전처리된 인구밀도를 standard scaler를 통해 정규 분포로 변환합니다.
scaler = StandardScaler()
dense['인구밀도'] = scaler.fit_transform(dense[['인구밀도']])
display(dense)

C:\Users\kyj93\AppData\Local\Temp\ipykernel_39724\3156907871.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dense['인구밀도'] = scaler.fit_transform(dense[['인구밀도']])


,행정동명,인구밀도
0,서울특별시 종로구 사직동,-1.366754
1,서울특별시 종로구 삼청동,-1.884942
2,서울특별시 종로구 부암동,-1.669245
3,서울특별시 종로구 평창동,-1.863621
4,서울특별시 종로구 무악동,-0.055429
...,...,...
421,서울특별시 강동구 천호2동,-0.128453
422,서울특별시 강동구 길동,-0.184864
423,서울특별시 강동구 강일동,-0.743028
424,서울특별시 강동구 상일제1동,-0.729081


- 공영주차장 데이터

In [11]:
parking = pd.read_csv('서울시 공영주차장.csv',encoding='euc-kr')
display(parking)

,주차장명,위도,경도
0,초안산근린공원주차장(구),37.649279,127.040474
1,마들스타디움(근린공원)(구),37.643917,127.058567
2,마장동(건물) 공영주차장(구),37.569939,127.035145
3,영등포여고 공영(구),37.515205,126.915116
4,당산근린공원 공영(구),37.525526,126.895794
...,...,...,...
1145,삼양마을공원지하 공영주차장(구),37.622745,127.018428
1146,색동공원 공영주차장(구),37.633227,127.020711
1147,비봉주차장(구),37.612180,126.956535
1148,신도림역환승주차장(구),37.507334,126.891653


- 지하철 승차 인원 데이터 전처리

In [12]:
df = pd.read_csv('지하철 승차.csv', encoding = 'utf-8') 
df = df.drop(columns = ['사용월', '호선명'], axis=1)

with open('station_coordinate.json', encoding='utf-8') as f:
    a = json.loads(f.read())
#불러온 데이터 프레임에서 컬럼을 위도, 경도, 지하철 역, 승하차 인원 수만 남깁니다.
js = pd.DataFrame(a)
js = js.drop(columns = ['line', 'code'], axis=1)
js = js.rename(columns = {'lat' : '위도', 'lng' : '경도', 'name' : '지하철역'})
js = js.drop_duplicates().reset_index(drop=True)
#지하철 역 별 유동인구의 평균을 구합니다.
subway = pd.merge(df, js, how = 'inner', on = '지하철역')
g_subway = subway.groupby(['지하철역']).agg('mean')
#시간대 별 유동인구를 나누기 위해 위도 경도를 잠시 빼고
lat = g_subway[['위도','경도']]
g_subway = g_subway.drop(columns = ['위도','경도'])
#stack을 통해 지하철 역 별 시간대 별 유동인구 형태로 만들고 
g_sub = pd.DataFrame(g_subway.stack())
g_sub = g_sub.rename(columns = {0 : '승차인원수'})
#위도 경도를 다시 붙힙니다.
g_sub1 = pd.merge(g_sub,lat, how = 'left', on = '지하철역')

#index에서 시간대만 뽑아 시간대 column을 만들어줍니다.
index = []
b = []
for i in g_sub.index:
    index.append(i[1])

for j in index:
    b.append(int(j[:2]))

g_sub1['시간대'] = b   

g_sub1 = g_sub1.dropna()
g_sub1.isnull().sum()

승차인원수    0
위도       0
경도       0
시간대      0
dtype: int64

- 지하철 하차 인원 데이터 전처리

In [13]:
#승차 인원 데이터 전처리와 동일합니다.
df = pd.read_csv('지하철 하차.csv', encoding = 'utf-8') 
df = df.drop(columns = ['사용월', '호선명'], axis=1)

with open('station_coordinate.json', encoding='utf-8') as f:
    a = json.loads(f.read())

js = pd.DataFrame(a)
js = js.drop(columns = ['line', 'code'], axis=1)
js = js.rename(columns = {'lat' : '위도', 'lng' : '경도', 'name' : '지하철역'})
js = js.drop_duplicates().reset_index(drop=True)

subway = pd.merge(df, js, how = 'inner', on = '지하철역')
g_subway = subway.groupby(['지하철역']).agg('mean')

lat = g_subway[['위도','경도']]
g_subway = g_subway.drop(columns = ['위도','경도'])

g_sub = pd.DataFrame(g_subway.stack())
g_sub = g_sub.rename(columns = {0 : '하차인원수'})

g_sub2 = pd.merge(g_sub,lat, how = 'left', on = '지하철역')

index = []
b = []
for i in g_sub.index:
    index.append(i[1])

for j in index:
    b.append(int(j[:2]))

g_sub2['시간대'] = b   

g_sub2 = g_sub2.dropna()
g_sub2.isnull().sum()

하차인원수    0
위도       0
경도       0
시간대      0
dtype: int64

In [14]:
#g_sub1.to_csv('승차인원수.csv', encoding = 'cp949', index = False)
#g_sub2.to_csv('하차인원수.csv', encoding = 'cp949', index = False)

- 서울 공공 풋살장 전처리

In [15]:
public_futsal = pd.read_csv("서울공공1.csv")
display(public_futsal)

,연번,구장명,주소,비고,위도,경도
0,1,금천구 풋살장,서울 금천구 가산동 562-2,"6:6, 야외",37.469092,126.882152
1,2,고척스카이돔풋살구장,서울 구로구 안양천로 545,"6:6, 야외",37.495817,126.867397
2,3,난우중학교풋살장,서울 관악구 난곡로16길 32,"6:6, 야외",37.466861,126.923310
3,4,오금풋살구장,서울 송파구 오금동 163,"6:6, 야외",37.498468,127.138511
4,5,중랑천제5체육공원풋살장,서울 중랑구 중화동 345-2,"6:6, 야외",37.605244,127.072198
5,6,초안산근린공원풋살장,서울 도봉구 창동 677,"6:6, 야외",37.648999,127.040280
6,7,응봉풋살장,서울 성동구 응봉동 240-1,"6:6, 야외",37.551178,127.036210
7,8,잠실종합운동장무브풋살파크,서울 송파구 올림픽로 25,"6:6, 야외",37.512190,127.075308
8,9,이촌한강공원풋살장,서울 용산구 한남동 782-2,"5:5, 야외",37.528390,127.007380
9,10,노들나루공원풋살장,서울 동작구 노량진로 247,"5:5, 야외",37.513509,126.954442


In [16]:
public_futsal['info'] = public_futsal['구장명'] + " | " + public_futsal['비고']

- 서울 민간 풋살장 전처리

In [17]:
private_futsal = pd.read_csv("서울민간1.csv")
display(private_futsal)

,연번,구장명,주소,비고,교육/대관,위도,경도
0,1,스트리트풋살파크,서울 강서구 화곡로 142 메가스퀘어빌딩 8층,"5:5, 옥상",대관,37.540567,126.837582
1,2,로꼬풋살아레나신목동점,서울 양천구 등촌로 100,"4:4, 실내",대관,37.538855,126.863957
2,3,한국풋살클럽,서울 양천구 등촌로 10 옥상 한국풋살클럽,"4:4, 옥상",교육,37.530823,126.864236
3,4,피치푸마목동,서울 양천구 목동서로 170 홈플러스 목동점 6F,"6:6, 옥상",대관,37.530244,126.873300
4,5,로꼬풋살아레나목동점,서울 양천구 목동서로 213,"4:4, 실내",대관,37.528018,126.870616
5,6,로꼬풋살아레나신도림점,"서울 구로구 신도림로11나길 8 지상, 지하 1층","4:4, 실내",대관,37.509084,126.879422
6,7,킥오프풋볼클럽,서울 영등포구 선유로 85 지하1층,"4:4, 실내",교육,37.521529,126.889530
7,8,업핏풋볼그라운드,서울 영등포구 당산로 241 9F,"4:4, 실내",교육,37.535215,126.902539
8,9,플랩스타디움가산마리오점,서울 금천구 벚꽃로 266 4층,"5:5, 옥상",대관,37.478446,126.885081
9,10,관악축구센터,서울 관악구 관악로 116 지하1층,"4:4, 실내",교육,37.475604,126.953083


In [18]:
private_futsal['info'] = private_futsal['구장명'] + " | " + private_futsal['비고'] + " | " + private_futsal['교육/대관']

In [19]:
#public_futsal.to_csv('public_futsal.csv', encoding = 'cp949', index = False)
#private_futsal.to_csv('private_futsal.csv', encoding = 'cp949', index = False)